In [1]:
import geopandas as gpd
import pandas as pd
from copy import deepcopy
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon, LineString, MultiLineString
from shapely.ops import unary_union
import numpy as np
from shapely.wkt import loads

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
gdf_hole = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/マンホール/yokohama_2018_2022.csv', encoding='cp932').filter(['人孔番号', 'geometry_x'])

<ipython-input-3-d782d924483b>:1: DtypeWarning: Columns (94,98,101,115) have mixed types. Specify dtype option on import or set low_memory=False.
  gdf_hole = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/マンホール/yokohama_2018_2022.csv', encoding='cp932').filter(['人孔番号', 'geometry_x'])


In [4]:
def parse_point(point_str):
    try:
        #文字列になっているから必要な数値だけ取得（POINT Z）を削除
        point_str = point_str[9:-2]

        #空白で分けx, yに代入
        x_str, y_str = point_str.strip().split()
        #ポイントに変換
        return Point(float(x_str), float(y_str))

        #エラーの場合は欠損地を入力
    except ValueError:
        return np.nan

#各行に対して関数を用いるときはapplyを用いる。Seriesだとエラーが起こることが多い
gdf_hole['geometry_x'] = gdf_hole['geometry_x'].apply(parse_point)

In [5]:
#dataframeをgeodataframeに変換
gdf_hole = GeoDataFrame(gdf_hole, geometry='geometry_x')

In [6]:
#空間結合した後にgeomeryを加えられるように連番を追加
number = [i for i in range(len(gdf_hole))]
gdf_hole['number'] = number

In [8]:
gdf_station = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/鉄道/N02-22_Station.shp', encoding='shift-jis')

In [9]:
if gdf_hole.crs is None:
    # 座標系をEPSG:6677に設定
    gdf_hole.set_crs(epsg=6677, inplace=True)

In [10]:
#座標系を変更
src_proj = 4326 # 変換前の座標系を指定
dst_proj = 6677  # 変換後の座標系を指定

gdf_station.crs = f'epsg:{src_proj}'
gdf_station = gdf_station.to_crs(epsg=dst_proj)

In [11]:
#線路のデータフレームのカラム名を変換
station_columns = {'N02_001': 'station_classification',
                   'N02_002': 'station_Business_Type',
                   'N02_003': 'station_route_name',
                   'N02_004':'station_operating_company',
                   'N02_005':	'station_name',
                   'N02_005c': 'station_code',
                   'N02_005g': 'station_group_code'}
gdf_station.rename(columns=station_columns, inplace=True)

In [12]:
#空間結合をする
result = gpd.sjoin_nearest(gdf_hole, gdf_station, how="left",lsuffix='point',rsuffix='road')
#空間結合された道路のgeometryを追加
result = pd.merge(result, gdf_station[['station_name', 'geometry']], on='station_name')
# #geometryの名前を変更
result.rename(columns={'geometry':'nearest_station'}, inplace=True)
#メモリの解放
del gdf_station

In [13]:
#dictance_railwyカラムを追加
result['distance_station'] = None

for i in range((len(result) // 50)+1):
    try:
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_station'][50*i:50*(i+1)]))

        # ポイントとマルチラインストリングとの最短距離を計算
        result['distance_station'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
        #一回ごとにメモリを解放
        del merged_lines
    except Exception as e:
        # エラーが発生した場合の処理
        print(f"Error occurred at the last batch: {e}")
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_station'][50*(i+1):]))

        result['distance_station'][50*(i+1):] = result.geometry_x[50*(i+1):].apply(lambda p: p.distance(merged_lines))

ストリーミング出力は最後の 5000 行に切り捨てられました。
<ipython-input-13-3ad9af224296>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_station'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-13-3ad9af224296>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_station'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-13-3ad9af224296>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [17]:
#重複を削除
result = result[~result.number.duplicated()]

#indexを並び替え
result.reset_index(inplace=True)

#numberはいらないのでdrop
result.drop(columns=['index', 'geometry_x', 'index_road', 'number'], axis=1, inplace=True)

<ipython-input-17-a1d512f6b130>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(columns=['index', 'geometry_x', 'index_road', 'number'], axis=1, inplace=True)


In [21]:
result_columns = result.columns.to_list()
result[result_columns].to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/横浜市　最近傍鉄道/横浜市_駅.csv', index=False)